In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/CurrentWork/Manifold/AdversarialRobustnessGeneralization

/content/drive/MyDrive/CurrentWork/Manifold/AdversarialRobustnessGeneralization


In [3]:
%pip install wandb==0.13.4
%pip install nvidia-ml-py3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
import os
import wandb
import pandas as pd
sys.path.append(os.getcwd())
import random
import numpy as np
import pandas as pd
# packages from https://github.com/irhete/predictive-monitoring-benchmark/blob/master/experiments/experiments.py
from DatasetManager import DatasetManager
from settings import global_setting, model_setting, training_setting
from LSTM import Model, CheckpointSaver
#user-specified packages
from util.DataCreation import DataCreation
from util.Arguments import Args
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import roc_auc_score
# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
# to explicitly raise an error with a stack trace to easier debug which operation might have created the invalid values
torch.autograd.set_detect_anomaly(True)

#hyperopt
import hyperopt
from hyperopt import hp, Trials, fmin, tpe, STATUS_OK
from hyperopt.pyll.base import scope
import yaml
# set logging
import logging
logging.getLogger().setLevel(logging.INFO)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)
##################################################################

dataset_ref_to_datasets = {
    "production": ["production"],
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(1,6)],
    "bpic2012": ["bpic2012_accepted","bpic2012_cancelled","bpic2012_declined"],
    "hospital_billing": ["hospital_billing_%s"%suffix for suffix in [2,3]],
    "traffic_fines": ["traffic_fines_%s" % formula for formula in range(1, 2)],
    # "bpic2017": ["bpic2017_accepted","bpic2017_cancelled","bpic2017_refused"],
    #"sepsis_cases": ["sepsis_cases_2","sepsis_cases_4"],
    #"bpic2011": ["bpic2011_f%s"%formula for formula in range(2,4)],
    }

dataset_name= 'bpic2015_1_f2'

path = 'params_dir_DL'
print('path',path)
os.environ['WANDB_DIR']= path
os.environ["WANDB_SILENT"] = "true"

dir_path = path+'/hyper_models/'+dataset_name
# create results directory
if not os.path.exists(os.path.join(dir_path)):
    os.makedirs(os.path.join(dir_path))

seed = global_setting['seed']
epochs = training_setting['epochs']
n_splits = global_setting['n_splits']
max_evals = global_setting['max_evals']
train_ratio = global_setting['train_ratio']
clip = training_setting["clip"]

project_name = 'Bayes_LSTM'
dataset_group = 'LSTM'
name = dataset_group + '_' + dataset_name

print('Dataset:', dataset_name)

dataset_manager = DatasetManager(dataset_name)
data = dataset_manager.read_dataset()
arguments = Args(dataset_name)
cls_encoder_args, min_prefix_length, max_prefix_length, activity_col, resource_col = arguments.extract_args(data, dataset_manager)
print(data.nunique())
cls_method = 'LSTM'
cls_encoding = 'embed'
datacreator = DataCreation(dataset_manager, dataset_name, cls_method, cls_encoding)
cat_cols = [activity_col, resource_col]
no_cols_list = []
cols = [cat_cols[0], cat_cols[1], cls_encoder_args['case_id_col'], 'label', 'event_nr', 'prefix_nr']
train, val = dataset_manager.split_data_strict(data, train_ratio, split="temporal")
train, test = dataset_manager.split_data_strict(train, train_ratio, split="temporal")
for i in cat_cols:
    _, _, _, no_cols = datacreator.create_indexes(i, train)
    no_cols_list.append(no_cols)
vocab_size = [no_cols_list[0]+1, no_cols_list[1]+1]
# you don't need to do that for the test data, as the prepare inputs is only fitted on the training data
# prepare chunks for CV
dt_prefixes = []

# prefix generation of test data
dt_train_prefixes = dataset_manager.generate_prefix_data(train, min_prefix_length, max_prefix_length)
dt_val_prefixes = dataset_manager.generate_prefix_data(val, min_prefix_length, max_prefix_length)
dt_test_prefixes = dataset_manager.generate_prefix_data(test, min_prefix_length, max_prefix_length)

#######WANDB################
with open("config/LSTM_sweep.yaml", 'r') as stream:
    sweep_config = yaml.safe_load(stream)

sweep_config['name'] =  name
sweep_id = wandb.sweep(sweep_config, project=project_name, entity="adversarial_robustness")

iterations= max_evals

train_cat_cols, val_cat_cols, _ = datacreator.prepare_inputs(dt_train_prefixes.loc[:, cat_cols], dt_val_prefixes.loc[:, cat_cols])
train_cat_cols, test_cat_cols, _ = datacreator.prepare_inputs(dt_train_prefixes.loc[:, cat_cols], dt_test_prefixes.loc[:, cat_cols])
dt_train_prefixes[cat_cols] = train_cat_cols
dt_val_prefixes[cat_cols] = val_cat_cols
dt_test_prefixes[cat_cols] = test_cat_cols
payload_values = {key: list(dt_train_prefixes[key].unique()) for key in cat_cols}
del train_cat_cols, test_cat_cols, val_cat_cols

dt_train_prefixes_original, dt_test_prefixes_original, dt_val_prefixes_original = dt_train_prefixes.copy(), dt_test_prefixes.copy(), dt_val_prefixes.copy()
nr_events_original = list(dataset_manager.get_prefix_lengths(dt_test_prefixes))
test_y_original = dataset_manager.get_label_numeric(dt_test_prefixes)
train_y_original = dataset_manager.get_label_numeric(dt_train_prefixes)
val_y_original = dataset_manager.get_label_numeric(dt_val_prefixes)
dt_train_prefixes = dt_train_prefixes[cols].copy()
dt_test_prefixes = dt_test_prefixes[cols].copy()
dt_val_prefixes = dt_val_prefixes[cols].copy()
train_y = dataset_manager.get_label_numeric(dt_train_prefixes)
test_y = dataset_manager.get_label_numeric(dt_test_prefixes)
val_y = dataset_manager.get_label_numeric(dt_val_prefixes)

# groupby case ID
ans_train_act = datacreator.groupby_caseID(dt_train_prefixes, cols, activity_col)
ans_test_act = datacreator.groupby_caseID(dt_test_prefixes, cols, activity_col)
ans_train_res = datacreator.groupby_caseID(dt_train_prefixes, cols, resource_col)
ans_test_res = datacreator.groupby_caseID(dt_test_prefixes, cols, resource_col)

######ACTIVITY_COL########
activity_train = datacreator.pad_data(ans_train_act).to(device)
activity_test = datacreator.pad_data(ans_test_act).to(device)

# ######RESOURCE COL########
resource_train = datacreator.pad_data(ans_train_res).to(device)
resource_test = datacreator.pad_data(ans_test_res).to(device)

def train(config=None):
	# Initialize a new wandb run
    with wandb.init(config=config,project=project_name, group=name, save_code=False) as run:
        global resource_train, resource_test, activity_train, activity_test
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        batch_size = config["batch_size"]
        hidden_size = config["hidden_size"]
        learning_rate = config['learning_rate']
        embed_size = training_setting['embed_size']

        print('embed_size', embed_size)
        print('hidden_size', hidden_size)
        print('batch_size', batch_size)
        print('learning rate', learning_rate)

        ###################MODEL ARCHITECTURE#################################################
        # create the input layers and embeddings
        dataset = torch.utils.data.TensorDataset(activity_train, resource_train)
        dataset = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True, worker_init_fn=seed_worker)

        model = Model(vocab_size, embed_size, hidden_size, max_prefix_length).to(device)
        print(model)
        if config['optimizer']=='RMSprop':
            optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
        elif config['optimizer']=='adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        elif config['optimizer']=='Nadam':
            optimizer = torch.optim.NAdam(model.parameters(), lr=learning_rate)
        print('optimizer', config['optimizer'])
        scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)
        # checkpoint saver
        checkpoint_saver = CheckpointSaver(dirpath=dir_path, decreasing=True, top_n=5)
        criterion = nn.BCELoss()
        # Define the early stopping patience
        print('training')
        early_start_patience = 15
        early_stop_patience = 60
        best_loss = 9999
        for epoch in range(epochs):
          print("Epoch: ", epoch)
          for i, (data_act, data_res) in enumerate(dataset, 0): # loop over the data, and jump with step = bptt.
                model.train()
                data_act = data_act.to(device)
                data_res = data_res.to(device)
                y_ = model(data_act,data_res).to(device)
                train_batch = torch.tensor(train_y[i*batch_size:(i+1)*batch_size], dtype= torch.float32).to(device)
                train_batch = train_batch[:, None].to(device)
                train_loss = criterion(y_,train_batch).to(device)
                wandb.log({"train_loss": train_loss})
                train_loss.backward()
                #torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
                optimizer.step()
                optimizer.zero_grad()

          with torch.no_grad():
            model.eval()
            #mask = ((activity_test != 0) | (resource_test != 0)).any(dim=1).float()
            print('testing')
            pred = model(activity_test, resource_test).squeeze(-1).to(device)

            validation_loss = criterion(pred, torch.tensor(test_y , dtype= torch.float32).to(device))
            validation_loss = validation_loss.to('cpu').detach().numpy()
            scheduler.step(validation_loss)
            # Log evaluation metrics to WandB
            wandb.log({"validation_loss":validation_loss})
            print('validation_loss',validation_loss)
            if epoch > early_start_patience:
              checkpoint_saver(model, epoch, validation_loss, learning_rate, hidden_size, config['optimizer'], batch_size)
            if validation_loss < best_loss:
              best_loss = validation_loss
          if epoch > early_stop_patience and validation_loss > best_loss:
                print('best loss', best_loss)
                print('validation loss', validation_loss)
                print("Early stopping triggered.")
                break
        # Empty CUDA cache
        torch.cuda.empty_cache()

wandb.agent(sweep_id, function= train, count=15, project=project_name, entity="adversarial_robustness")

path params_dir_DL
Dataset: bpic2015_1_f2
Responsible_actor                                       19
SUMleges                                               311
Case ID                                                696
label                                                    2
Aanleg (Uitvoeren werk of werkzaamheid)                  2
Bouw                                                     2
Brandveilig gebruik (melding)                            2
Brandveilig gebruik (vergunning)                         2
Gebiedsbescherming                                       2
Handelen in strijd met regels RO                         2
Inrit/Uitweg                                             2
Kap                                                      2
Milieu (melding)                                         2
Milieu (neutraal wijziging)                              2
Milieu (omgevingsvergunning beperkte milieutoets)        2
Milieu (vergunning)                                      2
Monument      

INFO:wandb.agents.pyagent:Starting sweep agent: entity=adversarial_robustness, project=Bayes_LSTM, count=15


embed_size 32
hidden_size 32
batch_size 128
learning rate 0.006496320018086848
Model(
  (embed_act): Embedding(242, 32)
  (embed_res): Embedding(17, 32)
  (lstm1): LSTM(64, 32, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
optimizer Nadam
training
Epoch:  0
testing
validation_loss 0.61655605
Epoch:  1
testing
validation_loss 0.5679024
Epoch:  2
testing
validation_loss 0.5405635
Epoch:  3
testing
validation_loss 0.52450055
Epoch:  4
testing
validation_loss 0.51560885
Epoch:  5
testing
validation_loss 0.51006263
Epoch:  6
testing
validation_loss 0.5066518
Epoch:  7
testing
validation_loss 0.50446403
Epoch:  8
testing
validation_loss 0.50331914
Epoch:  9
testing
validation_loss 0.5023379
Epoch:  10
testing
validation_loss 0.5017226
Epoch:  11
testing
validation_loss 0.5013082
Epoch:  12
testing
validation_loss 0.50107324
Epoch:  13
testing
validation_loss 0.5008533
Epoch:  14
testing
validation_loss 0.50072503
Epoch:  15
testing
valid

INFO:root:Current metric value better than 0.5005636215209961 better than best inf, saving model at params_dir_DL/hyper_models/bpic2015_1_f2/Model_0.006496320018086848_32_Nadam_128_epoch16.pt, & logging model weights to W&B.


testing
validation_loss 0.5005636
Epoch:  17


INFO:root:Current metric value better than 0.5005161166191101 better than best 0.5005636215209961, saving model at params_dir_DL/hyper_models/bpic2015_1_f2/Model_0.006496320018086848_32_Nadam_128_epoch17.pt, & logging model weights to W&B.


testing
validation_loss 0.5005161
Epoch:  18


INFO:root:Current metric value better than 0.5004550814628601 better than best 0.5005161166191101, saving model at params_dir_DL/hyper_models/bpic2015_1_f2/Model_0.006496320018086848_32_Nadam_128_epoch18.pt, & logging model weights to W&B.


testing
validation_loss 0.5004551
Epoch:  19
